**Notes:**

1. Loading data from PostgreSQL DB file

2. Creating custom functions to tidy up text data for analysis

## 1. Importing packages and Data Loadings

In [ ]:
# Importing necessary packages
import re
import os
import random, string
import pandas as pd
from string import punctuation
from google.colab import data_table

from sqlalchemy import create_engine
from sqlalchemy.sql import text
from sqlalchemy import inspect

data_table.enable_dataframe_formatter()

### 1.1 SQL DBs

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

In [ ]:
# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS uohyd_pgdaiml_project_db;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE uohyd_pgdaiml_project_db;'

NOTICE:  database "uohyd_pgdaiml_project_db" does not exist, skipping
DROP DATABASE
CREATE DATABASE


In [ ]:
%env TFIO_DEMO_DATABASE_NAME=uohyd_pgdaiml_project_db
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

env: TFIO_DEMO_DATABASE_NAME=uohyd_pgdaiml_project_db
env: TFIO_DEMO_DATABASE_HOST=localhost
env: TFIO_DEMO_DATABASE_PORT=5432
env: TFIO_DEMO_DATABASE_USER=postgres
env: TFIO_DEMO_DATABASE_PASS=postgres


In [ ]:
endpoint="postgresql://{}:{}@{}?port={}&dbname={}".format(
    os.environ['TFIO_DEMO_DATABASE_USER'],
    os.environ['TFIO_DEMO_DATABASE_PASS'],
    os.environ['TFIO_DEMO_DATABASE_HOST'],
    os.environ['TFIO_DEMO_DATABASE_PORT'],
    os.environ['TFIO_DEMO_DATABASE_NAME'],
)

In [ ]:
# PSQL engine and connection creations
my_psql_engine = create_engine(endpoint)
my_psql_conexion = my_psql_engine.connect()

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
!cp /content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/SQL_datos/los_mensajes_class_7726_db .

In [ ]:
# SQL DB Importing
!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d $TFIO_DEMO_DATABASE_NAME < los_mensajes_class_7726_db

In [ ]:
inspector = inspect(my_psql_engine)
schemas = inspector.get_schema_names()

In [ ]:
db_dict = dict({'relation_name':[],'no_of_records':[]})

In [ ]:
for r in my_psql_conexion.execute(text('SELECT schemaname,relname,n_live_tup FROM pg_stat_user_tables ORDER BY n_live_tup DESC')):
    db_dict['no_of_records'].append(r[2])
    db_dict['relation_name'].append(r[1])

In [ ]:
db_tables_df  = pd.DataFrame(db_dict)

In [ ]:
pd_df_names = []
for _ in range(6):
    pd_df_names.append(f'df_{_}')

In [ ]:
uohyd_sql_dataframes = [ ]
for df,table in zip(pd_df_names,db_dict['relation_name']):
    sql_query = text("SELECT *FROM {}".format(table))
    df = pd.read_sql(sql_query,my_psql_conexion)
    uohyd_sql_dataframes.append(df)

In [ ]:
messages_SC = pd.concat(uohyd_sql_dataframes)

## 2. Custom Functions for tidying text data

In [ ]:
def my_custom_url_check(texto):
    """Check if a message contains any URL or not"""
    txt_tidy = re.sub(r'\n|\r'," ",str(texto))
    rgx_url = re.compile('[a-zA-Z0-9]+([\-\.]{1}[a-zA-Z0-9]+)*\.[a-zA-Z]{2,5}(:[0-9]{1,5})?(\/.*)?$')
    return bool(re.search(rgx_url,str(txt_tidy)))

In [ ]:
def my_custom_url_extractor(texto):
    """Extract a URL from a message"""
    url_ext_reg = re.compile('((http(s)?\:\/\/)?[a-zA-Z0-9\@]+([\-\.]{1}[a-zA-Z0-9\@]+)*\.[a-zA-Z]{2,5}(:[0-9]{1,2})?((\/)[\w\d\!|\"|\#|\$|\%|\&|\'|\(|\)|\*|\+|\,|\-|\/|\:|\;|\<|\=|\>|\?|\[|\\|\]|\^|\_|\`|\{|\||\}|\~|\.]+)?)')
    urls = [ url[0] for url in url_ext_reg.findall(texto) ] 
    return urls

In [ ]:
def my_url_replacer(rgx,texto):
    """Replace an extracted URL position with a UNIQUE Keyword"""
    if len(rgx)>=2:
       patrn = '|'.join(map(re.escape, rgx))
       #return patrn
       return re.sub(patrn," MYURLEXTRACTED ",texto)
    elif len(rgx)==1:
        return re.sub(re.escape(rgx[0])," MYURLEXTRACTED ",texto)
    else:
        pass

In [ ]:
def my_string_tidy(texto):
    """Clean up all the punctuations from a message"""
    rgx_ltrs = re.compile("[\!|\"|\#|\$|\%|\&|\'|\(|\)|\*|\+|\,|\-|\/|\:|\;|\<|\=|\>|\?|\[|\\|\]|\^|\_|\`|\{|\||\}|\~|\.]+|FRM\:[\w\W]+(SUBJ\:)?MSG\:|\@|http(s)?|HTTP(S)?|\n|\r")
    return re.sub(rgx_ltrs,"",str(texto))

In [ ]:
def my_url_tidy(url):
    """Clean up an extracted URL Domain and collect a Main URL domain"""
    rgx_pat = r'http(s)?\:\/\/|(?<=[\w])\/([\w\W]+)?'
    # added \: and \? 
    rgx_url_short=r'[\w|\-]+\.[\w\:\?]+\Z'
    if len(url)>=2:
        url_more_two = []
        for item in url:
            url_more_two.extend(re.findall(rgx_url_short,re.sub(rgx_pat,"",str(item))))
        return [re.sub('\.','dot',u) for u in url_more_two]
    elif len(url)==1:
        url_list = re.findall(rgx_url_short,re.sub(rgx_pat,"",str(url[0])))
        return [ re.sub('\.','dot',item) for item in url_list ]
        return url_list
    else:
        pass
         

In [ ]:
def my_tidy_URL_replacer(text,replacer):
    """Replace an URL Unique Keyword with a tidy URL domain"""
    if len(replacer)>=2:
        rep_url = ' '.join(replacer)
        return re.sub('MYURLEXTRACTED',rep_url,text)
    elif len(replacer)==1:
        return re.sub('MYURLEXTRACTED',replacer[0],str(text))
    else:
        pass

In [ ]:
def my_custom_cta_email_check(texto):
    """Check if a message contains any EMAIL Call to Actions"""
    cta_rgx = r'(\b[A-Za-z-0-9\_\%\+\-.]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\s)'
    return bool(re.search(cta_rgx,str(texto)))

In [ ]:
def my_custom_cta_email_extract(texto):
    """Extract an EMAIL Call to Action from a message"""
    return re.sub(r'(\b[A-Za-z-1-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b)'," MYCTAEMAILEXTRACTED ",str(texto))

In [ ]:
def my_custom_cta_phone(texto):
    """Extract a PHONE NUMBER from message"""
    return re.sub(r'(1\s?)?((\([0-9]{3}\))|[0-9]{3})[\s\-]?[\0-9]{3}[\s\-]?[0-9]{4}(\:|\,|\.)?\s'," MYCTAPHONEEXTRACTED ",str(texto))

In [ ]:
def my_numerical_cleaner(texto):
    """Replace any digits with a unique keyword"""
    return re.sub(r'([\d]+(\s)?(\,)?(\-)?)?[\d]+'," DIGITEXTRACTED ",str(texto))

In [ ]:
def my_non_ascii(txt):
    """Check if a message contains any Non-ASCII words and replace them with a Unique Keywords"""
    return re.sub(r'[^\x00-\x7F]+'," NONASCII ",str(txt))

In [ ]:
my_stop_words_df = pd.read_excel('/content/drive/MyDrive/PG_Diploma_AI_ML_2021_UOHYD/PGDAIML_Project_Spam_Clustering/datos/my_smart_stop_words.xlsx')

my_stop_words_df_list = list(my_stop_words_df.stop_word)

In [ ]:
def my_custom_stop_word_removal(txt):
    stop_word_rgx = re.compile(r'\b(' + r'|'.join(my_stop_words_df_list) + r')\b\s*')
    return stop_word_rgx.sub('', txt)

In [ ]:
def my_custom_word_case_lower_upper(txt,caso="lower"):
    if caso =="lower":
        return txt.lower()
    else:
        return txt.upper()

## 3. Text Processings and Feature creations using above custom functions

In [ ]:
# Check a message contains any URL or not
messages_SC['has_URL'] = messages_SC['message_preview'].apply(
    lambda txt : my_custom_url_check(txt)
)

In [ ]:
# Check a message contains any EMAIL Call to Action or Not
messages_SC['has_EMAIL_CTA'] = messages_SC['message_preview'].apply(
    lambda txt : my_custom_cta_email_check(txt)
)

In [ ]:
# Create a new field URL DOMAIN to have an extracted URL from a message
messages_SC['URL_Domain'] = messages_SC.apply(
    lambda cols :  my_custom_url_extractor(cols['message_preview']) if ( (cols['has_URL'] is True) and (cols['has_EMAIL_CTA'] is False)) else None,axis=1
)

In [ ]:
# Removing unnecessary words of PH00 from messages
messages_SC['message_preview'] = messages_SC.message_preview.str.replace("\[\#\@\#PH00?[0-9]{1,2}\#\@\#\]","")

In [ ]:
# Replacing a URL with a UNIQUE KEYWORD: MYURLEXTRACTED
messages_SC['message_preview'] = messages_SC.apply(
    lambda cols: my_url_replacer(cols['URL_Domain'],cols['message_preview']) if ( (cols['has_URL'] is True) and (cols['has_EMAIL_CTA'] is False)) else cols['message_preview'],axis=1
    )

In [ ]:
# Find any CALL TO ACTION PHONE NUMBERS and Replace it with UNIQUE KEYWORD
messages_SC['message_preview'] = messages_SC['message_preview'].apply(lambda x: my_custom_cta_phone(x))

In [ ]:
# Remove all punctuations 
messages_SC['message_preview'] = messages_SC['message_preview'].apply(lambda x: my_string_tidy(x))

In [ ]:
# Remove not useful DIGITS
messages_SC['message_preview'] = messages_SC['message_preview'].apply(lambda x: my_numerical_cleaner(x))

In [ ]:
# Remove NON ASCII Words
messages_SC['message_preview'] = messages_SC['message_preview'].apply(lambda txt: my_non_ascii(txt))

In [ ]:
# Extract Main URL domain from an extracted URL part
messages_SC['url_tidy'] = messages_SC.apply(
    lambda cols: my_url_tidy(cols['URL_Domain']) if ( (cols['has_URL'] is True) and (cols['has_EMAIL_CTA'] is False)) else None,axis=1 
    )

In [ ]:
# Replace a URL keyword with an string based URL format like googledotcom or fbdotcom
messages_SC['message_preview'] = messages_SC.apply(
    lambda cols : my_tidy_URL_replacer(cols['message_preview'],cols['url_tidy']) if ( (cols['has_URL'] is True) and (cols['has_EMAIL_CTA'] is False)) else cols['message_preview'],axis=1
)

In [ ]:
# converting text to lower case letters
messages_SC['message_preview'] = messages_SC['message_preview'].apply(
    lambda msg : my_custom_word_case_lower_upper(msg)
)

In [ ]:
# Stop word removals
messages_SC['message_preview'] = messages_SC['message_preview'].apply(

    lambda msg : my_custom_stop_word_removal(msg)
)

In [ ]:
# random records checks on messages dataframe
messages_SC.iloc[4500:5000,:]

In [ ]:
# Save a tidy format messages dataframe into CSV 
messages_SC.to_csv('messages_classified_tidy_version_1.csv')